In [ ]:
import telebot
import time
import requests
import json
import threading
import random

from telebot import types

token = ''
bot = telebot.TeleBot(token)
users = []
data = {}
day_time = {}
lists = ['Приветики 💋', 'Ууиии какой сладосик 🥰', 'Маи харошие 🧡🤗🧡',
        'Ми-ми-ми 😍', 'Дай обниму 💕', 'Чмоки-чмоки в мокрый носик 🐾',
        'Шта??? Я не самый красивый 🥺', 'Ути-пути малышиии 🐣', 'Няшка-милашка 😍',
        'Р - Роскошный 😎', 'Не грусти пупсик 😘', 'Мррр 🐱', 'Гав 🐶']


@bot.message_handler(commands=['start']) 
def start_command(message):
    bot.send_message(message.from_user.id,
                f'Привет {message.from_user.first_name} ! \n' + 
                'Меня зовут Котопёс 😜 \n' +
                'И я с радостью разукрашу твои серые будни 🎊🎊🎊 \n' +
                'Одна картинка и ты улыбнешься! \n' +
                'Подписывайся скорей по команде /subscribe')
    
    
@bot.message_handler(commands=['subscribe'])
def get_subscribe(message):
    user_id = message.from_user.id
    if user_id not in users:
        users.append(user_id)
        
        keyboard = types.ReplyKeyboardMarkup(row_width=1, resize_keyboard=True)
        but1 = types.KeyboardButton('Кот 🐈')
        but2 = types.KeyboardButton('Собака 🐕')
        keyboard.add(but1, but2)
                         
        bot.send_message(message.from_user.id, 
                        f'{message.from_user.first_name} отлично! 😉 \n' +
                         'Скажи пожалуйста, кто тебе нравится больше \n' +
                         'кошки или собаки?',
                         reply_markup=keyboard)
                         
    else:
        bot.send_message(message.from_user.id,
                        {f'{message.from_user.first_name} Вы уже подписаны'})  
                

@bot.message_handler(commands=['unsubscribe'])
def get_unsubscribe(message):
    user_id = message.from_user.id
    if user_id not in users:
        bot.send_message(message.from_user.id,
                         f'Ошибка! {message.from_user.first_name} Вы не были подписаны! \n' +
                        'Подписка по команде /subscribe')
    else:
        users.remove(user_id)
        del data[user_id]
        del day_time[user_id]
        bot.send_message(message.from_user.id,
                         f'Подписка отменена! {message.from_user.first_name} спасибо что Вы были со мной! 💛')
            
            
@bot.message_handler(content_types=['text'])
def data_bot(message):
    user_id = message.from_user.id
    if message.text == 'Кот 🐈':
        for i in users:
            if i == user_id:
                data[user_id] = 'cat'
                day_time[user_id] = time.time()
       
        stiker = open('sticker_cat.webp', 'rb')
        bot.send_sticker(message.from_user.id, stiker)
        
        bot.send_message(message.from_user.id,
                        'Ок. И последнее, как только ты станешь достаточно радостным, \n' +
                        'можешь отписаться по команде /unsubscribe. \n' + 
                        'А сейчас, надо подождать и будет тебе порция дофамина! \n' + 
                        'До завтра! 😘')
        
    elif message.text == 'Собака 🐕':
        for i in users:
            if i == user_id:
                data[user_id] = 'dog'
                day_time[user_id] = time.time()
                
        stiker = open('sticker_dog.webp', 'rb')
        bot.send_sticker(message.from_user.id, stiker)

        bot.send_message(message.from_user.id,
                        'Ок. И последнее, как только ты станешь достаточно радостным, \n' +
                        'можешь отписаться по команде /unsubscribe. \n' + 
                        'А сейчас, надо подождать и будет тебе порция дофамина! \n' + 
                        'До завтра! 😘')
    
    else:
        bot.send_message(message.from_user.id,
                        'Ай донт андэстенд ю 😩')
        
        

def send_animals():
    while True:
        for user_id in users:
            print(user_id, users)
            if day_time[user_id] + 720 * 60 < time.time():
                if data[user_id] == 'cat':
                    bot.send_message(user_id, random.choice(lists)) 
                    bot.send_message(user_id, get_animals('cat')) 

                elif data[user_id] == 'dog':    
                    bot.send_message(user_id, random.choice(lists))
                    bot.send_message(user_id, get_animals('dog')) 
                     
                day_time[user_id] = time.time()
                
        time.sleep(60)
        
        
t = threading.Thread(target=send_animals)
t.start()      

        
def get_animals(animal):        
    if animal == 'cat':
        re = requests.get('https://api.thecatapi.com/v1/images/search')
        pic_cat = re.json()[0]['url']
        return pic_cat
    
    elif animal == 'dog':
        re = requests.get('https://api.thedogapi.com/v1/images/search')
        pic_dog = re.json()[0]['url']
        return pic_dog
    
    
    
bot.polling(none_stop=True)